In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import datetime
import random
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,mean_squared_error,r2_score
import wandb
from wandb.lightgbm import wandb_callback

In [2]:
os.environ["WANDB_NOTEBOOK_NAME"] = "hyperparameter_tuning_lightgbm_no_external_data.ipynb"

In [3]:
wandb.login()

wandb: Currently logged in as: cemalicoskunirmak (thebiasbusters). Use `wandb login --relogin` to force relogin


True

In [4]:
df = pd.read_excel("C:\\Users\\cemal\\Desktop\\Dersler\\Data Informatics\\DI502\\Github\\Electricity_Consmption_EPIAS_data.xlsx")

dates_dt = pd.to_datetime(df.Tarih, format='%d.%m.%Y')
    
df.Tarih = dates_dt.copy()
df = df.sort_values("Tarih")

In [5]:
df['year']=df['Tarih'].dt.year 
df['month']=df['Tarih'].dt.month
df['day']=df['Tarih'].dt.day
df['season']=df['Tarih'].dt.quarter
df['week']=df['Tarih'].dt.week
df['dayofweek']=df['Tarih'].dt.dayofweek
df['hour']= df.Saat.astype("str").apply(lambda x: x[:2]).astype("int")

df = df.sort_values(["year","month","day","hour"])

target = "Tüketim Miktarı (MWh)"

df.loc[:,target] =  pd.Series([item.replace(".", "").replace(",",".") for item in df.loc[:,target]]).astype("float")

C:\Users\cemal\AppData\Local\Temp\ipykernel_12736\1891689872.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['week']=df['Tarih'].dt.week
C:\Users\cemal\AppData\Local\Temp\ipykernel_12736\1891689872.py:13: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,target] =  pd.Series([item.replace(".", "").replace(",",".") for item in df.loc[:,target]]).astype("float")


In [6]:
train = df[df.Tarih <= "2023-09-30"].reset_index(drop=True).copy()
test = df[df.Tarih > "2023-09-30"].reset_index(drop=True).copy()

In [7]:
x_train = train.iloc[:,-7:]
x_test = test.iloc[:,-7:]
y_train = train.loc[:,target]
y_test = test.loc[:,target]

eval_set = [(x_test,y_test)]

In [8]:
sweep_config = {
    
    'method': 'bayes',  # Örneğin bayes, grid, random
    'metric': {
        
      'name': 'r2',
      'goal': 'maximize'
        
    },
    
    'parameters': {
        
        'learning_rate': {
            'min': 0.005,
            'max': 0.2
            
        },
        
        'max_depth': {
            'values': [4,5,6, 7]
            
        },
        
        'num_leaves': {
            'min': 20,
            'max': 50
        },
        
        'n_estimators': {
            'min':100,
            'max':1000
        },
        
        'subsample' : {
            'values' : [0.8,0.9,1]
        },
        
        'reg_alpha' : {
            
            'min': 0,
            'max':100
        },
        
        'reg_lambda' : {
            
            'min':0,
            'max':100
        }
    }
}

In [9]:
# sweep_id = wandb.sweep(sweep_config, project="ElectricConsumption", entity="thebiasbusters")

def train():
    # W&B entegrasyonunu başlat
    with wandb.init() as run:
        config = run.config

        # Modelinizi konfigürasyona göre oluşturun
        model = LGBMRegressor(
            learning_rate=config.learning_rate,
            max_depth=int(config.max_depth),
            num_leaves=int(config.num_leaves),
            n_estimators=int(config.n_estimators),
            subsample= config.subsample,
            reg_alpha= config.reg_alpha,
            reg_lambda= config.reg_lambda,
            random_state=42
        )

        # Modeli eğitin
        model.fit(x_train, y_train, eval_set=eval_set)
        preds = model.predict(x_test)

        # Calculate R-squared on the test set
        r2 = r2_score(y_test, preds)

        # Log the R-squared score
        wandb.log({'r2': r2})

Create sweep with ID: imyaimnq
Sweep URL: https://wandb.ai/thebiasbusters/ElectricConsumption/sweeps/imyaimnq


In [10]:
# Sweep agent'larını başlatın ve sweep ID'si ile ilişkilendirin
wandb.agent("imyaimnq", train, count=100)

wandb: Agent Starting Run: 12qzzqeb with config:
wandb: 	learning_rate: 0.12366309393238084
wandb: 	max_depth: 7
wandb: 	n_estimators: 477
wandb: 	num_leaves: 36
wandb: 	reg_alpha: 73
wandb: 	reg_lambda: 97
wandb: 	subsample: 0.9


r2,▁
r2,0.93305


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: orjyhnbf with config:
wandb: 	learning_rate: 0.0055539285987642115
wandb: 	max_depth: 5
wandb: 	n_estimators: 467
wandb: 	num_leaves: 32
wandb: 	reg_alpha: 6
wandb: 	reg_lambda: 45
wandb: 	subsample: 1


r2,▁
r2,0.88964


wandb: Agent Starting Run: 1cg2d49t with config:
wandb: 	learning_rate: 0.044719404156379934
wandb: 	max_depth: 6
wandb: 	n_estimators: 920
wandb: 	num_leaves: 48
wandb: 	reg_alpha: 97
wandb: 	reg_lambda: 72
wandb: 	subsample: 0.8


...

wandb: Agent Starting Run: aia6n6iu with config:
wandb: 	learning_rate: 0.17607320676790433
wandb: 	max_depth: 7
wandb: 	n_estimators: 920
wandb: 	num_leaves: 44
wandb: 	reg_alpha: 100
wandb: 	reg_lambda: 86
wandb: 	subsample: 0.9


r2,▁
r2,0.92672


wandb: Agent Starting Run: 0rx7pths with config:
wandb: 	learning_rate: 0.05975797090916945
wandb: 	max_depth: 7
wandb: 	n_estimators: 487
wandb: 	num_leaves: 30
wandb: 	reg_alpha: 52
wandb: 	reg_lambda: 70
wandb: 	subsample: 1


r2,▁
r2,0.90497


wandb: Ctrl + C detected. Stopping sweep.
